In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2,math,os,glob
r = plt.imread
def p(x):plt.imshow(x);plt.show()
import torch
from collections import OrderedDict 
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
def subplotter(img_list,ncols=6,figsize=14,names=None):
    nrows= math.ceil(len(img_list)/ncols)
    
    plt.figure(figsize=(figsize,figsize))
    for i,img in enumerate(img_list):
        plt.subplot(nrows,ncols,i+1)
        plt.imshow(img)
        if names:plt.title(names[i])
    plt.show()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../input/effb4initals')
print(glob.glob('*'))
w=torch.load('effb4-snapmix-epoch25-val_acc0.92.ckpt')['state_dict']
load = OrderedDict([( ('.').join(k.split('.')[1:]) ,v) for k, v in w.items()])

In [ ]:
os.chdir('/kaggle/input/pytorch-image-models/pytorch-image-models-master/')
import timm # resnest50d
model = timm.create_model('tf_efficientnet_b4_ns', pretrained=False,num_classes=5).cuda()
model.load_state_dict(load)
model.eval()
'loaded'


In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, 
    CLAHE, RandomRotate90, Transpose, ShiftScaleRotate, Blur, 
    OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur,
    IAAPiecewiseAffine, RandomResizedCrop, IAASharpen, IAAEmboss, 
    RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout,
    CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2, ToTensor


In [ ]:
input_size=380
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])


# val_augs=transforms.Compose([
#             transforms.ToPILImage(),
#             transforms.Resize((input_size,input_size)),
#             transforms.ToTensor(),
#             normalize ])
val_augs = Compose([
                        Resize(input_size, input_size),
                          Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                          ToTensorV2(),
                          ])

class TestPlantLoader(Dataset):
    def __init__(self, img_names, transform=None):
        self.img_names=img_names
        self.transform=transform
        
    def __len__(self):
        return len(self.img_names)
    def __getitem__(self, idx):
        
        img_name=self.img_names[idx]
        image=r(img_name)
        
        if self.transform: image = self.transform(image=image)['image']
        
        sample = {'image':image,'img_name':img_name}
        
        return sample

os.chdir('/kaggle/input/cassava-leaf-disease-classification/')
imgs = glob.glob('test_images/*') 

batch_size=1
dataset = TestPlantLoader(imgs,val_augs) 
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=8)


In [ ]:
ls=[]
for i, batch in enumerate(test_dataloader):
    x, name = batch['image'],batch['img_name']
    logits = model(x.cuda())
    preds = torch.argmax(logits, dim=1)
    
    ls.append([name[0].split('/')[-1],preds[0].item()])

    


In [ ]:
ls

In [ ]:
sub = pd.DataFrame.from_records(ls, columns=['image_id', 'label'])
sub.head()

In [ ]:
os.chdir('/kaggle/working/')
sub.to_csv("/kaggle/working/submission.csv", index=False)